## _*Quantum Battleships with Hardy's Paradox*_
For more information about how to use the IBM Q experience (QX), consult the [tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors
James R. Wootton
***

This is an alternate version of the game [Quantum Battleships with complementary measurements](https://github.com/decodoku/qiskit-sdk-py/blob/master/tutorial/sections/battleships_with_complementary_measurements.ipynb).

The game is based on the Japanese version of 'Battleships'. In this, each ship takes up only a single location. 

Each player will place two ships ships in the following five possible locations, which correspond to the five qubits of the ibmqx2 chip.

<pre>
                                                4       0
                                                |\     /|
                                                | \   / |
                                                |  \ /  |
                                                |   2   |
                                                |  / \  |
                                                | /   \ |
                                                |/     \|
                                                3       1
</pre>     

In this version of the game, two types of attack will be allowed: Bomb and Torpedo. But some ships can be immune to certain types of attack.

This is where it starts getting a bit quantum. Each ship can only be certain about whether or not it is immune to one type of attack. So if it is immune to bombs and you fire a torpedo, it will be random whether you attack succeeds or not. If it fails, the ship become certain that it is immune to torpedos. The effect of a bomb attack will then be random. By switching between attacks, you'll eventually destroy it.

The ships are made in the same shipyard, which means they have some correlations at the beginning. These correlations cause the ships to obey the following rules for the results of their first attack.

* They will never both repel a bomb attack.

* If one is destroyed by a torpedo attack, the other would repel a bomb attack.

These rules can be used to increase the chances of destroying both ships early on.

This game mechanic is realized on a quantum computer by using a qubit for each ship. Bomb attacks correspond to Z measurement, and torpedo attacks are X measurements. The outcome zero implies a failed attack (and so the ship is immune) and the outcome 1 implies that the ship has been destroyed. The correlations are acheived by initializing the qubits in an entangled state. The state used is one that corresponds to Hardy's paradox.

Some details on implementation can be found in the Markdown cells. A full tutorial for how to program the game is under construction.

<br>

If you just want to play, then select 'Restart & Run All' from the Kernel menu.
<br>
<br>
<br>
<br>

The next bunch of cells are largely the same as in the other *Battleships with...* games. The only difference is that two ships are chosen instead of three.

First we set up things required to run things using QISKit, and on the Quantum Experience.

Set up intructions can be found [here](https://github.com/IBM/qiskit-sdk-py/blob/master/tutorial/sections/tutorial4developer.ipynb).

In [1]:
import sys
sys.path.append("../../")
from qiskit import QuantumProgram
import Qconfig

Next we import a few standard things.

In [2]:
import getpass, random, numpy, math

The player is now asked to choose whether to run on the real device. The real device is awesome, of course, but you'll need to queue behind other people sampling its awesomeness. Any answer but *y* will simulate everything on your own (non-quantum) device.

In [3]:
d = input("Do you want to play on the real device? (y/n)\n")
if (d=="y"):
    device = 'ibmqx2'
else:
    device = 'local_qasm_simulator'
# note that device should be 'ibmqx_qasm_simulator', 'ibmqx2' or 'local_qasm_simulator'
# while we are at it, let's set the number of shots
shots = 1024

Do you want to play on the real device? (y/n)
n


It's good to break up games with a few 'Press Enter to continue...' type commands.

Here's the first, but it hides a secret! If you press r instead, the ships for both players will be randomly chosen.

In [4]:
randPlace = input("Press Enter to start placing ships...\n")

Press Enter to start placing ships...



The first step in the game is to get the players to set up their boards. Player 1 will be asked to give positions for three ships. Their inputs will be kept secret. Then the same for player 2.

In [5]:
numberOfShips = 2

# The variable ship[X][Y] will hold the position of the Yth ship of player X+1
shipPos = [ [-1]*numberOfShips for _ in range(2)] # all values are initialized to the impossible position -1|

# loop over both players and all three ships for each
for player in [0,1]:
    
    # if we chose to bypass player choice and do random, we do that
    if ((randPlace=="r")|(randPlace=="R")):
        randPos = random.sample(range(5), numberOfShips)
        for ship in range(numberOfShips):
            shipPos[player][ship] = randPos[ship]
        #print(randPos) #uncomment if you want a sneaky peak at where the ships are
    else:
        for ship in range(numberOfShips):

            # ask for a position for each ship, and keep asking until a valid answer is given
            choosing = True
            while (choosing):

                # get player input
                position = getpass.getpass("Player " + str(player+1) + ", choose a position for ship " + str(ship+1) + " (0, 1, 2, 3 or 4)\n" )

                # see if the valid input and ask for another if not
                if position.isdigit(): # valid answers  have to be integers
                    position = int(position)
                    if (position in [0,1,2,3,4]) and (not position in shipPos[player]): # they need to be between 0 and 5, and not used for another ship of the same player
                        shipPos[player][ship] = position
                        choosing = False
                        print ("\n")
                    elif position in shipPos[player]:
                        print("\nYou already have a ship there. Try again.\n")
                    else:
                        print("\nThat's not a valid position. Try again.\n")
                else:
                    print("\nThat's not a valid position. Try again.\n")

Player 1, choose a position for ship 1 (0, 1, 2, 3 or 4)
········


Player 1, choose a position for ship 2 (0, 1, 2, 3 or 4)
········


Player 2, choose a position for ship 1 (0, 1, 2, 3 or 4)
········


Player 2, choose a position for ship 2 (0, 1, 2, 3 or 4)
········




The main game loop is where the novel aspects of this game lie.

Each interation starts by asking players where on the opposing grid they want to attack, and what type of attack to use. The quantum computer then calculates the effects of the attacks, and the results are presented to the players. The game continues until all the ships of one player are destroyed.

In [6]:
# game variable will be set to False once the game is over
game = True

# states[player][ship] holds the state of each ship
states[player][ship] = [ [3]*numberOfShips for _ in range(2)]
# the state 0 is represented by +1, and 1 by -1
# the state + is represented by +2 and - by -2
# the initial state is represented by 3

# grid[player] will hold the results for the grid of each player
grid = [{},{}]

# attacked[player][position] tells you that this position of player's grid has ever been attacked
attacked = [ [0]*5 for _ in range(2)]

# damage has the amount of damage for each ship
damage = [ [0]*5 for _ in range(2)] # this will hold the prob of a 1 for each qubit for each player


while (game):
    
    input("Press Enter to start an attack...\n")
    
    # the variable bomb[X][Y] will hold the type of attack by X at position Y
    # 0 means none, 1 is bomb and 2 is torpedo
    bomb = [ [0]*5 for _ in range(2)] # all values are initialized to zero
    
    # ask both players where they want to attack and what kind of attack
    for player in [0,1]:
    
        print("\n\nIt's now Player " + str(player+1) + "'s turn.\n")

        # keep asking for position until a valid answer is given
        choosing = True
        while (choosing):
            # get player input
            position = input("Choose a position to attack (0, 1, 2, 3 or 4)\n")

            # see if this is a valid input. ask for another if not
            if position.isdigit(): # valid answers  have to be integers
                position = int(position)
                if position in [0,1,2,3,4]: # they need to be between 0 and 5, and not used for another ship of the same player
                    
                    choosing = False
                    print ("\n")
                else:
                    print("\nThat's not a valid position. Try again.\n")
            else:
                print("\nThat's not a valid position. Try again.\n")
    
        # keep asking for attack type until a valid answer is given
        choosing = True
        while (choosing):
            # get player input
            attack = input("Do you want to bomb or torpedo it? (b or t)\n")
            # see if this is a valid input. ask for another if not
            if attack  in ["b","t"]: # they need to be between 0 and 5, and not used for another ship of the same player
                choosing = False
                print ("\n")
            else:
                print("\nThat's not a valid attack type. Try again.\n")
        # record the attack type in 'bomb'
        if (attack=="b"):
            bomb[player][position] = 1
        else:
            bomb[player][position] = 2
        # and make sure 'attacked' knows that an attack as happened
        attacked[player][position] = 1
    
    # now we create and run the quantum programs that implement this on the grid for each player
    for player in range(2):
    
        # create a dictionary with the specifications of the program
        # we'll use all 5 qubits and bits, to avoid bugs on IBM's end
        Q_SPECS = {
            "circuits": [{
                "name": "gridScript",
                "quantum_registers": [{
                    "name": "q",
                    "size": 5
                }],
                "classical_registers": [{
                    "name": "c",
                    "size": 5
                }]}],
        }

        print("\nWe'll now get the quantum computer to see what happens to Player " + str(player+1) + "'s ships.\n")

        # create the program with these specs
        Q_program = QuantumProgram(specs=Q_SPECS)

        # get the circuit by name
        gridScript = Q_program.get_circuit("gridScript")
        # get the quantum register by name
        q = Q_program.get_quantum_registers("q")
        # get the classical register by name
        c = Q_program.get_classical_registers("c")
        
        # initialize the qubits
        if (states[player][0]==3) or (states[player][1]==3):
            makeHardyState(gridScript,q,states[player])
        else:   
            for ship in range(2):
                if (states[player][ship]==-1): # -1 => 1
                    gridScript.x(q[ship])
                elif (states[player][ship]==2): # 2 => +
                    gridScript.h(q[ship])
                elif (states[player][ship]==-2): # -2 => -
                    gridScript.x(q[ship])
                    gridScript.h(q[ship])
        
        
        # prepare rotations for the measurements if torpedos are used
        for ship in range(2):
            if (bomb[(player+1)%2][position]==2): # rotate to X basis
                gridScript.h(q[position])
                                        
        #finally, measure them
        for position in range(5):
            gridScript.measure(q[position], c[position])
            
        # to see what the quantum computer is asked to do, we can print the QASM file
        # these lines are typically commented out
        #QASM_source = Q_program.get_qasm("gridScript")
        #print(QASM_source)
        
        # set the APIToken and API url
        Q_program.set_api(Qconfig.APItoken, Qconfig.config["url"])
        # compile and run the QASM
        Q_program.execute(["gridScript"], device, shots, wait=2, timeout=60)
        
        # extract data
        grid[player] = Q_program.get_counts("gridScript")
    
    
    if ( ( 'Error' in grid[0].values() ) or ( 'Error' in grid[1].values() ) ):

        print("\nThe process timed out. Try this round again.\n")
        
    else:
    
        # look at the damage on all qubits (we'll even do ones with no ships)
        # for this we loop over all 5 bit strings for each player
        for player in range(2):
            for bitString in grid[player].keys():
                # and then over all positions
                for position in range(5):
                    # if the string has a 1 at that position, we add a contribution to the damage
                    if (bitString[position]=="1"):
                        damage[player][position] += grid[player][bitString]/shots
            
        # give results to players
        for player in [0,1]:

            input("\nPress Enter to see the results for Player " + str(player+1) + "'s ships...\n")

            # we can print the damage before further processing to check all is as it should be
            #print(damage[player])
            
            # we see which result is in the majority and use that
            display = [" ?  "]*5
            # loop over all qubits
            for position in shipPos[player]:
                # display a percentage damage if the ship has ever been attacked
                if (attacked[player][position]>0):

                        # mitigate the effects of errors by not allowing damages <5% or >95%
                        if (damage[player][position] < 0.05):
                            damage[player][position] = 0
                        elif (damage[player][position] > 0.95):
                            damage[player][position] = 1

                        # the damage is randomly reset to 1 or 0, using the current value as the probability
                        if ( random.random() < damage[player][position] ):
                            damage[player][position] = 1
                            display[position] = "100%"
                            states[player][position] = -bomb[(player+1)%2][position]
                        else:
                            damage[player][position] = 0
                            display[position] = " 0% "
                            states[player][position] = bomb[(player+1)%2][position]
                            
                            
            # we can print the states after the bombing to check all is as it should be
            #print(states[player])
                            
            print("Here is the percentage damage for ships that have been attacked.\n")
            print(display[ 4 ] + "    " + display[ 0 ])
            print(" |\     /|")
            print(" | \   / |")
            print(" |  \ /  |")
            print(" |  " + display[ 2 ] + " |")
            print(" |  / \  |")
            print(" | /   \ |")
            print(" |/     \|")
            print(display[ 3 ] + "    " + display[ 1 ])
            print("\n")
            print("Ships with 100% damage have been destroyed\n")
            print("Ships with 0% were immune to the attack\n")
            print("\n")

            # if a player has all their ships destroyed, the game is over
            # ideally this would mean 100% damage, but we go for 95% because of noise again
            if (damage[player][ shipPos[player][0] ]>.95) and (damage[player][ shipPos[player][1] ]>.95)):
                print ("***All Player " + str(player+1) + "'s ships have been destroyed!***\n\n")
                game = False

        if (game is False):
            print ("***Game Over***\n\n")
        

Press Enter to start an attack...



It's now Player 1's turn.

Choose a position to attack (0, 1, 2, 3 or 4)
0


Do you want to bomb or torpedo it? (b or t)
b




It's now Player 2's turn.

Choose a position to attack (0, 1, 2, 3 or 4)
0


Do you want to bomb or torpedo it? (b or t)
t



We'll now get the quantum computer to see what happens to Player 1's ships.

>> quantum_registers created: q 5
>> classical_registers created: c 5
running on backend: local_qasm_simulator

We'll now get the quantum computer to see what happens to Player 2's ships.

>> quantum_registers created: q 5
>> classical_registers created: c 5
running on backend: local_qasm_simulator

Press Enter to see the results for Player 1's ships...

Here is the percentage damage for ships that have been attacked.

 ?       0% 
 |\     /|
 | \   / |
 |  \ /  |
 |   ?   |
 |  / \  |
 | /   \ |
 |/     \|
 ?       ?  


Ships with 100% damage have been destroyed

Ships with 0% were immune to the attack




Press Enter to se

IndexError: list index out of range

<br>
<br>
<br>
If you are reading this while running the game, you might be wondering where all the action has gone. Try scrolling to the bottom of the output in the cell above.